In [1]:
# Import
import pathlib
import pretty_midi
import glob
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import mido

k = 5
n_song = 20

In [9]:
data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

In [10]:

import glob
filenames = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))

Number of files: 1282


In [11]:
sample_file = filenames[2]
pm = pretty_midi.PrettyMIDI(sample_file)

songs = [pretty_midi.PrettyMIDI(file) for file in filenames[:10]]

In [12]:
_SAMPLING_RATE = 16000
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [13]:
print('Number of instruments:', len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print('Instrument name:', instrument_name)

Number of instruments: 1
Instrument name: Acoustic Grand Piano


In [14]:
for i, note in enumerate(instrument.notes[:10]):
  note_name = pretty_midi.note_number_to_name(note.pitch)
  duration = note.end - note.start
  print(f'{i}: pitch={note.pitch}, note_name={note_name},'
        f' duration={duration:.4f}')

0: pitch=77, note_name=F5, duration=0.1042
1: pitch=73, note_name=C#5, duration=0.1003
2: pitch=68, note_name=G#4, duration=0.0664
3: pitch=49, note_name=C#3, duration=0.3424
4: pitch=73, note_name=C#5, duration=0.0872
5: pitch=77, note_name=F5, duration=0.0859
6: pitch=73, note_name=C#5, duration=0.0664
7: pitch=61, note_name=C#4, duration=0.1771
8: pitch=78, note_name=F#5, duration=0.0703
9: pitch=73, note_name=C#5, duration=0.0742


In [3]:
def pitch_to_midi(pitch_sequence):
    tempo = mido.bpm2tempo(120)
    time_signature = (4, 4, 24, 8)

    # Create a MIDI file and add the tempo and time signature information
    mid = mido.MidiFile()
    mid.ticks_per_beat = 480
    mid.tracks.append(mido.MidiTrack())
    mid.tracks[0].append(mido.MetaMessage("set_tempo", tempo=tempo))
    mid.tracks[0].append(mido.MetaMessage("time_signature", numerator=time_signature[0], denominator=time_signature[1], clocks_per_click=time_signature[2], notated_32nd_notes_per_beat=time_signature[3]))

    # Convert the pitch sequence to a list of MIDI notes
    notes = []
    for pitch in pitch_sequence:
        notes.append(mido.Message("note_on", note=pitch, velocity=100, time=0))
        notes.append(mido.Message("note_off", note=pitch, velocity=100, time=480))

    # Add the notes to the MIDI file
    mid.tracks[0].extend(notes)

    # Save the MIDI file
    mid.save("pitch_sequence.mid")


In [16]:
def pitch_to_array(pitch):
    array = np.zeros((128, 1))
    array[pitch] = 1
    return array

In [17]:
def song_to_data(start, notes):
    x = np.zeros((k, 128))
    y = np.zeros((1, 128))
    for i in range(start, start+k):
        x[i-start,notes[i]] = 1
    y[0, notes[start+k]] = 1
    return (x, y)


In [18]:
song = pretty_midi.PrettyMIDI("pitch_sequence.mid")
inst = song.instruments[0]
notes = [notes.pitch for i, notes in enumerate(inst.notes)]
x, y = song_to_data(0, notes)

print(x, y)



[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [20]:
x_train = np.zeros((0, k, 128))
y_train = np.zeros((0, 1, 128))

for song in songs[:n_song]:
    inst = song.instruments[0]
    notes = [note.pitch for i, note in enumerate(inst.notes)]
    start = 0
    print(len(notes))
    while start+k < len(notes):
        x, y = song_to_data(start, notes)
        x_train = np.append(x_train, x.reshape((1, k, 128)), axis=0)
        y_train = np.append(y_train, y.reshape((1, 1, 128)), axis=0)
        start += 1



2235
4231


KeyboardInterrupt: 

In [6]:
print(x_train.shape)
print(y_train.shape)

print(x_train[0, 0])
print(y_train[0])

NameError: name 'x_train' is not defined

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(k, 128)),
        layers.LSTM(128),
        layers.Dense(128, activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
Total params: 148,096
Trainable params: 148,096
Non-trainable params: 0
_________________________________________________________________


In [5]:
epochs = 40
batch_size = 128

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

NameError: name 'x_train' is not defined

In [20]:
m =[39, 27, 51, 42, 51, 30, 33, 39, 45, 51, 34, 46, 39, 51, 39, 47, 51, 35, 48, 39, 51, 36, 39, 33, 45, 51, 39, 30, 42, 51, 39, 27, 51, 42, 30, 51, 33, 39, 45, 51, 66, 67, 51, 50, 78, 71, 43, 39, 46, 76, 43, 53, 55, 53, 59, 68, 46, 53, 41, 38, 44, 68, 41, 38, 68, 38, 50, 38, 69, 66, 40, 67, 34, 38, 62, 35, 38, 38, 66, 49, 66, 40, 43, 67, 37, 67, 66, 50, 78, 67, 67, 50, 57, 58, 80, 69, 43, 67, 50, 57, 67, 69, 70, 53, 67, 50, 57, 38, 47, 35, 55, 47, 48, 69, 57, 69, 52, 51, 53, 56, 52, 40, 72, 51, 49, 69, 50, 67, 45, 38, 57, 69, 49, 61, 70, 67, 50, 60, 69, 70]
m2 = [39, 27, 51, 42, 51, 30, 33, 39, 45, 51, 34, 46, 39, 51, 39, 47, 51, 35, 48, 39, 51, 36, 39, 33, 45, 51, 39, 30, 42, 51, 39, 27, 51, 42, 30, 51, 33, 39, 45, 51, 34, 39, 46, 51, 39, 47, 51, 35, 48, 39, 51, 36, 39, 33, 45, 51, 39, 30, 42, 51, 39, 27, 51, 42, 30, 54, 33, 45, 57, 34, 46, 58, 35, 47, 48, 59, 36, 48, 50, 60, 37, 49, 61, 54, 62, 57, 49, 50, 55, 63, 58, 38, 66, 57, 60, 49, 50, 70, 65, 62, 38, 69, 60, 63, 50, 49, 62, 58, 67, 38, 60, 57, 66, 49, 50, 67, 61, 58, 38, 54, 62, 50, 55, 58, 63, 38, 57, 60, 66, 49, 50, 65, 62, 70, 38, 69, 60, 63, 49, 50]

In [22]:
pitch_to_midi(m2)